<a href="https://colab.research.google.com/github/bolulka/NeuralNetworks/blob/main/lab10/lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лаб. 10.1
Задание 1.  (из  тем  10.1–10.2):

Обучите рекуррентную нейронную сеть распознаванию тональности отзывов на тестовых данных открытого набора данных imdb.

Задание 2.  (из  тем  10.1–10.2):

С использованием предварительно обученной нейронной сети определите тональность своего отзыва.

In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

np.random.seed(42)

max_features = 5000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

maxlen = 80

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

model = Sequential()
model.add(Embedding(max_features, 32))
'''
50 - 82.46%
125 - 83.04%
150 - 83.45%
'''
model.add(LSTM(150, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

'''
adam - 83.08%
sgd - 50.15%
'''
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

'''
5 - 83.08%
10 - 82.42%
15 - 82.23%
'''
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_test, y_test), verbose=1)

scores = model.evaluate(X_test, y_test, batch_size=64)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))
#83.66%

model_json = model.to_json()
json_file = open("model.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model.h5")

In [12]:
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import text
from keras.preprocessing import sequence

json_file = open("model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")

loaded_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

maxlen = 80
review = 'a pretty nice movie to watch with family'
words = text.text_to_word_sequence(review)
dim = len(words)

x = text.hashing_trick(review, dim, hash_function='md5')
x = np.expand_dims(x, axis=0)
x = sequence.pad_sequences(x, maxlen=maxlen)

prediction = loaded_model.predict(x, verbose=0)
print(prediction)


[[0.8960314]]


In [ ]:
import keras
from keras.datasets import imdb

NUM_WORDS = 5000   # only use top 5000 words
INDEX_FROM = 3   # word index offset

(train_x, train_y), (test_x, test_y) = keras.datasets.imdb.load_data(
        num_words=NUM_WORDS, index_from=INDEX_FROM)

word_to_id = imdb.get_word_index()
word_to_id = {k: (v+INDEX_FROM) for k, v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value: key for key, value in word_to_id.items()}

review_id = 25
print(' '.join(id_to_word[num] for num in train_x[review_id]))
